In [1]:
from langchain.prompts import ChatPromptTemplate, PromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate, AIMessagePromptTemplate
from langchain_openai.chat_models import ChatOpenAI

In [2]:
from langchain.output_parsers import DatetimeOutputParser

In [4]:
output_parser = DatetimeOutputParser()

In [45]:
template_text = "{request}\n{format_instructions}"
human_prompt = HumanMessagePromptTemplate.from_template(template_text)

# OPTION 1: One way to make sure output always has a correct format is to use a system prompt
# system_prompt = SystemMessagePromptTemplate.from_template("You only respond to questions with a singe datetime timestamp")

# HACK: In order to demonstrate the output fixing parswer, I  need to make sure something invalid is returned
#       so adding FOO: as a prefix to the output so that it is an invalid timestamp
system_prompt = SystemMessagePromptTemplate.from_template("You always add FOO: followed by a datetime timestamp when responding to questions")

In [46]:
chat_prompt = ChatPromptTemplate.from_messages([system_prompt, human_prompt])

In [47]:
model_request = chat_prompt.format_prompt(request="What date was the 13th amendment ratified in the US?",
                          format_instructions=output_parser.get_format_instructions()).to_messages()

In [48]:
model = ChatOpenAI()

In [72]:
result = model.invoke(model_request)

In [74]:
# output_parser.parse(result.content)

In [57]:
misformatted = result.content

In [58]:
from langchain.output_parsers import OutputFixingParser

fixing_parser = OutputFixingParser.from_llm(parser=output_parser, llm=model)

In [71]:
fixed_result = fixing_parser.parse(misformatted)
fixed_result

datetime.datetime(1865, 12, 6, 0, 0)